In [ ]:
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
import re
import time
import pandas as pd
from matplotlib import pyplot as plt
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver
from nltk.corpus import stopwords
import nltk
import os
print(os.getcwd())

nltk.download('stopwords')

In [ ]:
#Get Job id to get clickable element on indeed, needed to extract the full jobdescription
def get_jobid(what, country):
    total_job = 500  # Job limit
    jobid = []
    seen = set(jobid)
    prev_job_num = 0
    
    for i in range(0, total_job, 10):
        # Construct search result URLs
        web = f'https://www.indeed.com/jobs?q={what}&l={country}&start={i}'
        web_base = f'https://www.indeed.com/jobs?q={what}&l={country}'
        
        # Set up Chrome options
        options = Options()
        options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
        options.add_argument('--headless')  # Optional: Run in headless mode
        options.add_argument('--disable-gpu')
        options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
        options.add_argument('--no-sandbox')            # Bypass OS security model
        options.add_argument('--remote-debugging-port=9222')  # Ensure stable debugging connection


        # Initialize WebDriver
        driver = webdriver.Chrome(options=options)
        wait = WebDriverWait(driver, timeout=10)
        
        try:
            # Fetch the webpage
            driver.get(web)
            src = driver.page_source
        finally:
            driver.quit()
        
        # Extract the total number of jobs on the first iteration
        if i == 0:
            try:
                total_jobs_line = next(line for line in src.splitlines() if 'name="description"' in line)
                num_job = int(re.search(r'\d+', re.sub("[^a-zA-Z0-9]", "", total_jobs_line)).group())
                print(f"{num_job} jobs available for '{what}' in {country} on Indeed.com.")
            except Exception as e:
                print("Could not extract total job count:", e)
                num_job = total_job  # Default to total_job if extraction fails

        # Extract job keys from `window._initialData`
        try:
            job_keys = re.findall(r'"jobKey":"(.*?)"', src)  # Regex to find all job keys
            for jid in job_keys:
                if jid not in seen:
                    seen.add(jid)
                    jobid.append(jid)
        except Exception as e:
            print("Error extracting job keys:", e)
        
        # Stop if we've found all jobs or no new jobs were added
        if len(jobid) >= num_job or len(jobid) == prev_job_num:
            print(f"Number of (non-repeated) jobs found = {len(jobid)}")
            break
        prev_job_num = len(jobid)
    
    return jobid, web_base

In [ ]:
def get_text(website):
    # Set up Chrome options
    options = Options()
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36')
    # options.add_argument('--headless')  # Uncomment for headless mode
    options.add_argument('--disable-gpu')
    options.add_argument('--headless')
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems
    options.add_argument('--no-sandbox')            # Bypass OS security model
    options.add_argument('--remote-debugging-port=9222')  # Ensure stable debugging connection

    
    # Initialize WebDriver
    driver = webdriver.Chrome(options=options)
    driver.get(website)
    print("Page Title:", driver.title)
    
    content = ""
    
    try:
        # Wait for the job description text to appear
        wait = WebDriverWait(driver, 30)
        job_description = wait.until(EC.presence_of_element_located((By.ID, 'jobDescriptionText')))
        content = job_description.text  # Extract the job description text
    except Exception as e:
        print(f"Error while extracting job description: {e}")
    finally:
        driver.quit()
    
    return content

In [ ]:
def get_job_desc(jobid, web_base):
    # Generate URLs for job descriptions
    urls = [web_base + '&vjk=' + str(job) for job in jobid]
    print("Generated URLs:", urls)

    job_desc = []
    for website in urls:
        # Extract job description for each URL
        desc = get_text(website)
        if desc:  # Only add non-empty descriptions
            job_desc.append(desc)
            print("Extracted Job Description:", desc)
    
    print('Number of (non-repeated) jobs found =', len(job_desc))
    return job_desc

In [ ]:
# Verify desirable output
web_base = "https://www.indeed.com/jobs?q=large+language+models&l=United+States&from=searchOnDesktopSerp"
job_ids = ["44b6c858cc3704bd", "60e7238f01971f93"]  # Example job IDs
descriptions = get_job_desc(job_ids, web_base)

# Print the extracted descriptions
for idx, desc in enumerate(descriptions, 1):
    print(f"\nJob {idx} Description:\n{desc}")

In [ ]:
descriptions

In [ ]:
jobid, web_base = get_jobid('Large+Language+Models','United+States')
job_desc = get_job_desc(jobid, web_base)
data = count_tools(job_desc)

In [ ]:
job_desc

In [ ]:
data

In [ ]:
# Save the list as a text file
with open("job_descriptions.txt", "w", encoding="utf-8") as file:
    file.write("\n".join(job_desc))  # Each item on a new line

In [ ]:
search_terms = [
    "Large Language Models", "Foundation Models", "Generative AI", "Transformer Models",
    "LLM Engineer", "AI Engineer", "NLP Engineer", "Prompt Engineer",
    "AI Research Scientist", "Applied AI", "Language Model Specialist"
]
location = "United States"

# Initialize an empty list to store all job descriptions
all_job_desc = []

for term in search_terms:
    # Replace spaces with '+' for URL encoding
    what = term.replace(' ', '+')
    # Call the get_jobid function for each term
    job_ids, web_base = get_jobid(what, location)
    # Extract job descriptions for the job IDs
    job_desc = get_job_desc(job_ids, web_base)
    # Add descriptions to the aggregated list
    all_job_desc.extend(job_desc)

# Location for saving
file_path = "/Users/moisejanssens/Desktop/job_desc_LLM.txt"

# Save the aggregated job descriptions to the Desktop
with open(file_path, "w", encoding="utf-8") as file:
    file.write("\n".join(all_job_desc))  # Each description on a new line

print(f"Aggregated job descriptions saved to '{file_path}'.")